# What this file does

I didn't really end up using this — I created a map of municipalities and income inequality that I used to lay under my buyouts map. It was a very busy map, and geographically representing the data was probably not the best way forward. I used American Community Survey data and the GINI index (https://www.nj.com/news/2017/02/how_every_town_in_nj_scores_on_income_inequality.html#:~:text=According%20to%20the%20latest%20numbers,of%200.619%2C%20the%20Census%20shows) and did some joins with a municipal codes database (https://www.state.nj.us/treasury/taxation/pdf/lpt/cntycode.pdf). This notebook just joins different datasets and makes sure municipal codes are attached to relevant values for better joins in QJIS.

# Making it easier to map this data

In [ ]:
#import libraries 

In [1]:
import pandas as pd

In [ ]:
#read in income data, ACS 2011-2015, data from census bureau

In [2]:
df = pd.read_csv('income_ACS11-15_mun.csv')

In [ ]:
#for joins - convert to upper

In [32]:
df['county2'] = df.county.apply(lambda x: to_upper(x))

In [28]:
def to_upper(name):
    try:
        return name.upper()
    except:
        print(name)
        pass

In [ ]:
#checking for format of names

In [44]:
df['name'].apply(lambda x: check_t(x)).value_counts()

False    513
True      49
Name: name, dtype: int64

In [38]:
def check_t(x):
    return 'Township' in x

In [46]:
def check_b(x):  
    return 'Borough' in x

In [48]:
def check_c(x):  
    return 'City' in x

In [47]:
df['name'].apply(lambda x: check_b(x)).value_counts()

False    558
True       4
Name: name, dtype: int64

In [49]:
df['name'].apply(lambda x: check_c(x)).value_counts()

False    549
True      13
Name: name, dtype: int64

In [ ]:
#the issue is that lots of these have these suffixes, which I don't want for the purposes of the join later. Best to find municial codes

In [4]:
#GINI index by county; data from https://www.nj.com/news/2017/02/how_every_town_in_nj_scores_on_income_inequality.html#:~:text=According%20to%20the%20latest%20numbers,of%200.619%2C%20the%20Census%20shows.

In [58]:
df2 = pd.read_csv('gini_by_county.csv')

In [59]:
df2['county_upper'] = df2.county.apply(lambda x: to_upper(x))

In [5]:
#save the version

In [60]:
df2.to_csv('gini_county_new.csv', index=False)

In [62]:
df['name2'] = df.name.apply(lambda x: to_upper(x))

In [63]:
df

,name,county,median_income,county2,name2
0,North Caldwell,Essex,200152,ESSEX,NORTH CALDWELL
1,Millburn,Essex,184783,ESSEX,MILLBURN
2,Essex Fells,Essex,172386,ESSEX,ESSEX FELLS
3,Harding Township,Morris,171331,MORRIS,HARDING TOWNSHIP
4,Glen Ridge,Essex,170433,ESSEX,GLEN RIDGE
...,...,...,...,...,...
557,Wildwood,Cape May,27067,CAPE MAY,WILDWOOD
558,Salem,Salem,26320,SALEM,SALEM
559,Atlantic City,Atlantic,25737,ATLANTIC,ATLANTIC CITY
560,Camden,Camden,25042,CAMDEN,CAMDEN


In [ ]:
#grabbed New Jersey Municipal codes: https://www.state.nj.us/treasury/taxation/pdf/lpt/cntycode.pdf

In [67]:
df3 = pd.read_csv('mun_codes.csv')

In [77]:
df3

,CODE,MUNICIPALITY,COUNTY
0,101,ABSECON,Atlantic County
1,102,ATLANTIC CITY,Atlantic County
2,103,BRIGANTINE,Atlantic County
3,104,BUENA,Atlantic County
4,105,BUENA VISTA,Atlantic County
...,...,...,...
560,2119,PHILLIPSBURG,Warren County
561,2120,POHATCONG,Warren County
562,2121,WASHINGTON,Warren County
563,2122,WASHINGTON,Warren County


### df

In [6]:
#saving it

In [80]:
df.set_index('name2').join(df3.set_index('MUNICIPALITY')).to_csv('income_with_codes.csv')
